# Implementation Naive Bayes Spam Filtering from scratch

### Author: Sunwoo Choi

### Data
reference: https://github.com/justmarkham/DAT5/blob/master/data/SMSSpamCollection.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import re
data = np.loadtxt(fname='/content/drive/MyDrive/Colab Notebooks/ML/SMSSpamCollection.txt', dtype=str, delimiter='\t')

data_size = data.shape[0]

# Set seed and shuffle
np.random.seed(1337)
np.random.shuffle(data)



In [ ]:
data = data.tolist()

# cleaning up data set
# 1. delete all punctuations
# 2. split the sting into words
for i in range(data_size):
  data[i][1] = re.sub(r'[^A-Za-z0-9\']+', ' ',data[i][1].lower())
  data[i][1] = data[i][1].split()

split_idx = (int) (len(data)*0.7)
print(split_idx)

3901


In [ ]:
# splite the whole data set to 70% training set and 30% validation set
train_set = data[:split_idx]
test_set = data[split_idx:]

print(len(train_set)+len(test_set))

5574


In [ ]:
# Dictionary to check number of total, non-spam, spam message
entire = { "ham":0, "spam":0, "total":0 } 
# Word counting dictionary in spam/non-spam message
ham_word, spam_word = {}, {}
# Total number of non-spam/ spam/ unique words
total_ham_word, total_spam_word, total_unique_word = 0, 0, 0


# Check every word in the training set and counting number of words
for i in range(len(train_set)):
  entire[train_set[i][0]] += 1
  for word in train_set[i][1]:
    if train_set[i][0] == "ham":
      if word in ham_word:
        ham_word[word] = ham_word[word] + 1
      else:
        ham_word[word] = 1
        total_unique_word += 1
      total_ham_word += 1
    else:
      if word in spam_word:
        spam_word[word] = spam_word[word] + 1
      else:
        spam_word[word] = 1
        total_unique_word += 1
      total_spam_word += 1

entire["total"] = len(train_set)

print(entire)
print(ham_word)
print(spam_word)
print(total_unique_word)

{'ham': 3386, 'spam': 515, 'total': 3901}
{"wherre's": 1, 'my': 533, 'boytoy': 12, 'oh': 81, 'lk': 1, 'tt': 4, 'den': 25, 'we': 207, 'take': 84, 'e': 65, 'one': 119, 'ends': 7, 'at': 272, 'cine': 4, 'lor': 118, 'dun': 40, 'wan': 39, 'yogasana': 4, 'oso': 16, 'can': 279, 'dunno': 20, 'cos': 57, 'i': 1589, 'was': 145, 'v': 42, 'late': 37, 'n': 109, 'when': 195, 'reach': 17, 'they': 76, 'inside': 5, 'already': 59, 'but': 311, 'ate': 6, 'spageddies': 1, "it's": 67, 'gals': 5, 'who': 40, 'r': 99, 'laughing': 4, 'me': 573, 'good': 162, 'afternoon': 19, 'sexy': 10, 'buns': 4, 'how': 186, 'goes': 18, 'the': 763, 'job': 29, 'search': 13, 'wake': 13, 'and': 568, 'you': 1289, 'are': 297, 'first': 40, 'thought': 33, 'as': 110, 'always': 36, 'love': 138, 'wish': 37, 'your': 287, 'fine': 31, 'happy': 70, 'know': 166, 'adore': 3, 'spoke': 6, 'with': 187, 'uncle': 10, 'john': 10, 'today': 100, 'he': 136, 'strongly': 1, 'feels': 8, 'that': 347, 'need': 105, 'to': 1026, 'sacrifice': 2, 'keep': 47, 'here

In [ ]:
# probabilty of receiving non spam message
p_ham = entire['ham']/entire['total']
# probabilty of receiving spam message
p_spam = entire['spam']/entire['total'] 

print(p_ham)
print(p_spam)

0.8679825685721609
0.132017431427839


In [ ]:
num_of_success = 0
total_test = 0
alpha = 1 # slack variable

# testing the model

for i in range(len(test_set)):
  label = test_set[i][0]
  prob_ham, prob_spam = p_ham, p_spam # P(spam), P(ham)
  for word in test_set[i][1]:
    num_ham = ham_word[word] if word in ham_word else 0
    num_spam = spam_word[word] if word in spam_word else 0
    prob_ham *= (num_ham + alpha)/(total_ham_word+total_unique_word) # P(word|ham)
    prob_spam *= (num_spam+ alpha)/(total_spam_word+total_unique_word) # P(word|spam)

  pred_label = 'ham' if prob_ham > prob_spam else 'spam' 
  if label == pred_label: # If prediction succeed, increase number of success by 1.
    num_of_success += 1
  total_test += 1 

print(f"The accuracy of naive bayes classifier is {num_of_success/total_test*100}%")  

The accuracy of naive bayes classifier is 98.08726838015541%
